# Check Attributs column to filter Restaurants rather than other business

In [ ]:
import pandas as pd

# yelp datasets file
yelp_datasets_path = './yelp_datasets/'

# yelp dataset (from yelp offical website)
yelp_offical_dataset_path = yelp_datasets_path + 'yelp_dataset_official/'

# business, user and review dataset
yelp_academic_dataset_business_path = yelp_offical_dataset_path + 'yelp_academic_dataset_business.json'
yelp_academic_dataset_user_path = yelp_offical_dataset_path + 'yelp_academic_dataset_user.json'
yelp_academic_dataset_review_path = yelp_offical_dataset_path + 'yelp_academic_dataset_review.json'

# yelp photo dataset (from yelp official website)
yelp_offical_photo_dataset_path = yelp_datasets_path + 'yelp_dataset_official_photos/'

# photo dataset
photo_dataset_path = yelp_offical_photo_dataset_path + 'photos/'

def check_attributes_and_filter_out_restaurants(yelp_academic_dataset_business_path, business_type='restaurant'):
    
    import re
    
    # Set chunk size
    chunk_size = 10000
    
    # Initialize an empty DataFrame to store the sampled results
    filtered_businesses = pd.DataFrame()
    
    if business_type=='restaurant':
        # Define keywords related to restaurants
        restaurant_keywords = [
            'restaurant', 'food', 'drink', 'cafe', 'dining', 'eatery', 'bistro', 'diner', 'tavern',
            'brasserie', 'trattoria', 'eatery', 'pizzeria', 'sushi', 'grill', 'deli', 'bakery',
            'pub', 'steakhouse', 'barbecue', 'buffet', 'noodle', 'rice', 'pasta', 'spaghetti', 'pizza',
            'burger', 'sandwich', 'taco', 'burrito', 'sushi', 'sashimi', 'ramen', 'pho', 'curry',
            'fried chicken', 'schnitzel', 'kebab', 'shawarma', 'pad thai', 'fried rice', 'poke bowl',
            'soup', 'salad', 'sushi roll', 'spring roll', 'gyro', 'hot dog', 'pancake', 'waffle',
            'crepe', 'bagel', 'toast', 'omelette', 'scrambled eggs', 'frittata', 'quiche', 'pita',
            'wrap', 'sandwich', 'sub', 'sourdough', 'muffin', 'croissant', 'biscuit', 'hash browns',
            'fries', 'onion rings', 'samosa', 'dim sum', 'bao', 'empanada', 'pierogi', 'lasagna',
            'casserole', 'risotto', 'gnocchi', 'meatball', 'stew', 'chili', 'paella', 'souvlaki',
            'tandoori', 'tempura', 'schnitzel', 'satay', 'katsu', 'bibimbap', 'tostada', 'enchilada',
            'tortilla', 'quesadilla', 'poutine', 'syrup', 'gravy', 'curry', 'hummus', 'tzatziki',
            'guacamole', 'salsa', 'pesto', 'tapenade', 'sauce', 'condiment'
        ]


        # Iterate through the data using chunks
        for chunk in pd.read_json(yelp_academic_dataset_business_path, lines=True, encoding='utf-8-sig', chunksize=chunk_size):
            # Convert categories to lowercase
            chunk['categories'] = chunk['categories'].str.lower()
            # Drop rows with NaN values in categories column
            chunk.dropna(subset=['categories'], inplace=True)
            # Split categories into individual words
            chunk['category_words'] = chunk['categories'].apply(lambda x: re.findall(r'\b\w+\b', x))
            # Filter businesses containing restaurant keywords
            filtered_chunk = chunk[chunk['category_words'].apply(lambda x: any(word in restaurant_keywords for word in x))]
            # Append filtered chunk to the result DataFrame
            filtered_businesses = pd.concat([filtered_businesses, filtered_chunk], ignore_index=True)

    return filtered_businesses

In [ ]:
# restaurants = check_attributes_and_filter_out_restaurants(yelp_academic_dataset_business_path, business_type='restaurant')
# restaurants

In [ ]:
# restaurants.to_csv("restaurants_businesses.csv", index=False)

In [ ]:
def filter_reviews_for_certain_business_type(yelp_academic_dataset_review_path, businesses):
    # Set chunk size
    chunk_size = 10000

    # Initialize an empty DataFrame to store the sampled results
    filtered_business_reviews = pd.DataFrame()

    # Iterate through the data using chunks
    for chunk in pd.read_json(yelp_academic_dataset_review_path, lines=True, encoding='utf-8-sig', chunksize=chunk_size):
        # Filter reviews for restaurants
        filtered_chunk = chunk[chunk['business_id'].isin(businesses['business_id'])]
        # Append filtered chunk to the result DataFrame
        filtered_business_reviews = pd.concat([filtered_business_reviews, filtered_chunk], ignore_index=True)

    return filtered_business_reviews

In [ ]:
# import pandas as pd

# restaurants = pd.read_csv("restaurants_businesses.csv")
# filtered_restaurants_reviews = filter_reviews_for_certain_business_type(yelp_academic_dataset_review_path, restaurants)
# filtered_restaurants_reviews

In [ ]:
# # try to shuffle the whole restaurant review data
# filtered_restaurants_reviews_shuffled = filtered_restaurants_reviews.sample(frac=1, random_state=42).reset_index(drop=True)
# filtered_restaurants_reviews_shuffled

In [ ]:
# filtered_restaurants_reviews_shuffled.to_csv("restaurants_reviews_shuffled.csv", index=False)

# Sample reviews and calculate price then return data

In [5]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

# yelp datasets file
yelp_datasets_path = './yelp_datasets/'

# yelp dataset (from yelp offical website)
yelp_offical_dataset_path = yelp_datasets_path + 'yelp_dataset_official/'

# business, user and review dataset
yelp_academic_dataset_business_path = yelp_offical_dataset_path + 'yelp_academic_dataset_business.json'
yelp_academic_dataset_user_path = yelp_offical_dataset_path + 'yelp_academic_dataset_user.json'
yelp_academic_dataset_review_path = yelp_offical_dataset_path + 'yelp_academic_dataset_review.json'

# yelp photo dataset (from yelp official website)
yelp_offical_photo_dataset_path = yelp_datasets_path + 'yelp_dataset_official_photos/'

# photo dataset
photo_dataset_path = yelp_offical_photo_dataset_path + 'photos/'

# Download the nltk punkt tokenizer
# nltk.download('punkt') # already downloaded once

# Function to calculate token count for a text using nltk
def calculate_token_count_nltk(text):
    tokens = word_tokenize(text)
    return len(tokens)

def sample_reviews_and_calculate_price_then_return_data(data, percent, reviews_num_threshold, genai="GPT-3.5 Turbo", sampling_method='random', column='None'):
    
    print("Percent:", percent, "%")
    print("Threshold reviews:", reviews_num_threshold)
    print("Generative AI model:", genai)
    print("Sampling_method:", sampling_method)
    print("Column:", column)
    print("\n")
    
    sampled_percent_reviews = pd.DataFrame()
    
    if sampling_method=='random':
        
        sampled_percent_reviews = data.sample(frac=(percent * 0.01), random_state=42)
    
    elif sampling_method=='stratified' and column!='None':
        
        # Calculate the occurrence count of the column
        column_value_counts = data[column].value_counts()
        # Calculate the total number data
        total_num = len(data)
        # Calculate the ratio of each star rating
        column_value_ratios = column_value_counts / total_num
        
        for value, ratio in column_value_ratios.items():
            sampled_subset = data[data[column] == value].sample(frac=(percent * 0.01), random_state=42)
            sampled_percent_reviews = pd.concat([sampled_percent_reviews, sampled_subset], ignore_index=True)
        
    else:
        print("Please specify sample type and column!")

    # Print the sampled results
    sampled_percent_reviews_num = len(sampled_percent_reviews)
#     print(f"Number of {percent}% reviews:", sampled_percent_reviews_num)
    
    # Select reviews from the sampled dataset based on the filtering conditions
    filtered_reviews = sampled_percent_reviews
    
    while True:

        user_review_count = filtered_reviews['user_id'].value_counts()
        business_review_count = filtered_reviews['business_id'].value_counts()
        excluded_users = user_review_count[user_review_count < reviews_num_threshold].index.values
        excluded_businesses = business_review_count[business_review_count < reviews_num_threshold].index.values

        if (len(excluded_users) == 0) and (len(excluded_businesses) == 0):
            break
        else:
            selected_users = user_review_count[user_review_count >= reviews_num_threshold].index.values
            selected_businesses = business_review_count[business_review_count >= reviews_num_threshold].index.values
            filtered_reviews = filtered_reviews[(filtered_reviews['user_id'].isin(selected_users)) & (filtered_reviews['business_id'].isin(selected_businesses))]
            # print(len(filtered_reviews))

    # print(n)
    # print(filtered_reviews)
    filtered_reviews_num = len(filtered_reviews)
#     print("Number of filtered_reviews:", filtered_reviews_num)
    
    filtered_users_count = filtered_reviews['user_id'].nunique()
    filtered_businesses_count = filtered_reviews['business_id'].nunique()

#     print('Number of unique users in filtered reviews:', filtered_users_count)
#     print('Number of unique businesses in filtered reviews:', filtered_businesses_count)
#     print('\n')

    # Calculate min, average, max reviews per user
    min_reviews_per_user = filtered_reviews.groupby('user_id')['review_id'].count().min()
    mean_reviews_per_user = filtered_reviews.groupby('user_id')['review_id'].count().mean()
    max_reviews_per_user = filtered_reviews.groupby('user_id')['review_id'].count().max()

    # Calculate min, average, max reviews per business
    min_reviews_per_business = filtered_reviews.groupby('business_id')['review_id'].count().min()
    mean_reviews_per_business = filtered_reviews.groupby('business_id')['review_id'].count().mean()
    max_reviews_per_business = filtered_reviews.groupby('business_id')['review_id'].count().max()

    # Print the results
#     print('Min reviews per user:', min_reviews_per_user)
#     print('Mean reviews per user:', mean_reviews_per_user)
#     print('Max reviews per user:', max_reviews_per_user)
#     print('\n')
#     print('Min reviews per business:', min_reviews_per_business)
#     print('Mean reviews per business:', mean_reviews_per_business)
#     print('Max reviews per business:', max_reviews_per_business)
#     print('\n')
    
#     GPT-3.5 Turbo: The price for one English token input is 0.000015 TWD, and one English token output is 0.000045 TWD.
#     GPT-4 Turbo (20 times the price): The price for one English token input is 0.0003 TWD, and one English token output is 0.0009 TWD.
#     GPT-4 Turbo Image: The price for a 1080 x 1080 image is 0.2295 TWD.
#     GPT-4 (40 times the price): The price for one English token input is 0.0006 TWD, and one English token output is 0.0018 TWD.
#     GPT-4 32k (80 times the price): The price for one English token input is 0.0012 TWD, and one English token output is 0.0036 TWD.
    
    model_NT_price_per_token_output = 0.000045
    model_NT_price_per_token_input = (model_NT_price_per_token_output / 3)
    model_NT_price_per_token_input_and_output = ((model_NT_price_per_token_input + model_NT_price_per_token_output) / 2)
    model_NT_price_per_picture = 0.2295
    
    if genai=="GPT-3.5 Turbo":
        model_NT_price_per_token_output = 0.000045
        model_NT_price_per_token_input = (model_NT_price_per_token_output / 3)
        model_NT_price_per_token_input_and_output = ((model_NT_price_per_token_input + model_NT_price_per_token_output) / 2)
  
    elif genai=="GPT-4 Turbo":
        model_NT_price_per_token_output = 0.0009
        model_NT_price_per_token_input = (model_NT_price_per_token_output / 3)
        model_NT_price_per_token_input_and_output = ((model_NT_price_per_token_input + model_NT_price_per_token_output) / 2)
  
    elif genai=="GPT-4 Turbo Image":
        model_NT_price_per_picture = 0.2295
        
    elif genai=="GPT-4":
        model_NT_price_per_token_output = 0.0018
        model_NT_price_per_token_input = (model_NT_price_per_token_output / 2)
        model_NT_price_per_token_input_and_output = ((model_NT_price_per_token_input + model_NT_price_per_token_output) / 2)
  
    elif genai=="GPT-4 32k":
        model_NT_price_per_token_output = 0.0036
        model_NT_price_per_token_input = (model_NT_price_per_token_output / 2)
        model_NT_price_per_token_input_and_output = ((model_NT_price_per_token_input + model_NT_price_per_token_output) / 2)
  
    else:
        print("Please specify Generative AI type or correct type!")

    # Apply the function to calculate token count for each review
    filtered_reviews['token_count_nltk'] = filtered_reviews['text'].apply(calculate_token_count_nltk)

    # Calculate average token count per user and per business
    total_token_count_per_user = filtered_reviews.groupby('user_id')['token_count_nltk'].sum()
    avg_token_count_per_user = total_token_count_per_user.mean()
    total_token_count_per_business = filtered_reviews.groupby('business_id')['token_count_nltk'].sum()
    avg_token_count_per_business = total_token_count_per_business.mean()

    # Print the results
#     print('Average Token Count per User (nltk):', avg_token_count_per_user)
#     print('Average Token Count per Business (nltk):', avg_token_count_per_business)
#     print('\n')

    # Calculate price
    
    price_per_user_by_output = 2 * avg_token_count_per_user * model_NT_price_per_token_output
    price_per_user_by_input_and_output = 2 * avg_token_count_per_user * model_NT_price_per_token_input_and_output
    price_for_all_user_by_output = filtered_users_count * price_per_user_by_output
    price_for_all_user_by_input_and_output = filtered_users_count* price_per_user_by_input_and_output
    
#     print(f"{genai} Price per user by output:", price_per_user_by_output)
#     print(f"{genai} Price per user by input and output:", price_per_user_by_input_and_output)
#     print(f"{genai} Price for all user by output:", price_for_all_user_by_output)
#     print(f"{genai} Price for all user by input and output:", price_for_all_user_by_input_and_output)
#     print('\n')
    
    price_per_business_by_output = 2 * avg_token_count_per_business * model_NT_price_per_token_output
    price_per_business_by_input_and_output = 2 * avg_token_count_per_business * model_NT_price_per_token_input_and_output
    price_for_all_business_by_output = filtered_businesses_count * price_per_business_by_output
    price_for_all_business_by_input_and_output = filtered_businesses_count * price_per_business_by_input_and_output
    
#     print(f"{genai} Price per business by output:", price_per_business_by_output)
#     print(f"{genai} Price per business by input and output:", price_per_business_by_input_and_output)
#     print(f"{genai} Price for all business by output:", price_for_all_business_by_output)
#     print(f"{genai} Price for all business by input and output:", price_for_all_business_by_input_and_output)

#     print("\n")
    print("-----------------------------------------------------------")
    print("\n")
    
    calculation_results = {}
    calculation_results["percent"] = percent
    calculation_results["reviews_num_threshold"] = reviews_num_threshold
    calculation_results["sampling_method"] = sampling_method
    calculation_results["column"] = column
    calculation_results["sampled_percent_reviews_num"] = sampled_percent_reviews_num
    calculation_results["filtered_reviews_num"] = filtered_reviews_num
    calculation_results["filtered_users_count"] = filtered_users_count
    calculation_results["filtered_businesses_count"] = filtered_businesses_count
    calculation_results["min_reviews_per_user"] = min_reviews_per_user
    calculation_results["mean_reviews_per_user"] = mean_reviews_per_user
    calculation_results["max_reviews_per_user"] = max_reviews_per_user
    calculation_results["min_reviews_per_business"] = min_reviews_per_business
    calculation_results["mean_reviews_per_business"] = mean_reviews_per_business
    calculation_results["max_reviews_per_business"] = max_reviews_per_business
    calculation_results["genai"] = genai
    calculation_results["model_NT_price_per_token_input"] = model_NT_price_per_token_input
    calculation_results["model_NT_price_per_token_output"] = model_NT_price_per_token_output
    calculation_results["model_NT_price_per_token_input_and_output"] = model_NT_price_per_token_input_and_output
    calculation_results["model_NT_price_per_picture"] = model_NT_price_per_picture
    calculation_results["avg_token_count_per_user"] = avg_token_count_per_user
    calculation_results["avg_token_count_per_business"] = avg_token_count_per_business
    calculation_results["price_per_user_by_output"] = price_per_user_by_output
    calculation_results["price_per_user_by_input_and_output"] = price_per_user_by_input_and_output
    calculation_results["price_for_all_user_by_output"] = price_for_all_user_by_output
    calculation_results["price_for_all_user_by_input_and_output"] = price_for_all_user_by_input_and_output
    calculation_results["price_per_business_by_output"] = price_per_business_by_output
    calculation_results["price_per_business_by_input_and_output"] = price_per_business_by_input_and_output
    calculation_results["price_for_all_business_by_output"] = price_for_all_business_by_output
    calculation_results["price_for_all_business_by_input_and_output"] = price_for_all_business_by_input_and_output
    
    # return the filtered reviews and calculation results
    return filtered_reviews, calculation_results
#     return calculation_results

In [3]:
def combination_results_list_to_df(all_combination_results):
    
    import pandas as pd

    all_combination_results_df = pd.DataFrame(columns = all_combination_results[0].keys())

    for calculation_result in all_combination_results:
        all_combination_results_df = pd.concat([all_combination_results_df, pd.DataFrame([calculation_result])], ignore_index=True)

    return all_combination_results_df

In [4]:
def filter_combination_results(combination_calculation_results_df, price_limit = 300, reviews_num = 5000, user_num = 500, business_num = 500):

    combination_filtered_df = combination_calculation_results_df[
        ((combination_calculation_results_df["price_for_all_business_by_output"] <= price_limit) | (combination_calculation_results_df["price_for_all_business_by_input_and_output"] <= price_limit))
        & (combination_calculation_results_df["filtered_reviews_num"] >= reviews_num)
        & (combination_calculation_results_df["filtered_users_count"] >= user_num)
        & (combination_calculation_results_df["filtered_businesses_count"] >= business_num)
    ]

    return combination_filtered_df

## Calculate all combination results (Sampling method is random)

In [ ]:
# # sampling by random
# all_combination_results = []

# percent_list = [i for i in range(1, 21)]
# reviews_num_threshold_list = [i for i in range(1, 11)]
# genai_list = ["GPT-3.5 Turbo"]
# sampling_method_list = ['random']
# column_list = ["stars"]

# filtered_restaurants_reviews_shuffled = pd.read_csv("restaurants_reviews_shuffled.csv")

# for percent in percent_list:
#     for reviews_num_threshold in reviews_num_threshold_list:
#         for genai in genai_list:
#             for sampling_method in sampling_method_list:
#                 for column in column_list:
#                     _, calculation_results = sample_reviews_and_calculate_price_then_return_data(filtered_restaurants_reviews_shuffled, percent=percent, reviews_num_threshold=reviews_num_threshold, genai=genai, sampling_method=sampling_method, column=column)
#                     all_combination_results.append(calculation_results)

In [ ]:
# all_combination_results

In [ ]:
# all_combination_results_df = combination_results_list_to_df(all_combination_results)
# all_combination_results_df

In [ ]:
# all_combination_results_df.to_csv("../../research related documents/random_sampling_restaurant_reviews_combination_calculation_results.csv", index=False)

In [ ]:
# import pandas as pd

# price_limit = 300
# reviews_num = 5000
# user_num = 500
# business_num = 500

# combination_calculation_results_df = pd.read_csv("../../research related documents/random_sampling_restaurant_reviews_combination_calculation_results.csv")

# filter_combination_results(combination_calculation_results_df, 
#                            price_limit, reviews_num, user_num, business_num)

In [ ]:
# import os
# import pandas as pd

# percent = 13
# reviews_num_threshold = 5
# genai = "GPT-3.5 Turbo"
# sampling_method = 'random'
# column = "stars"

# filtered_restaurants_reviews_shuffled = pd.read_csv("restaurants_reviews_shuffled.csv")
# df, calculation_results = sample_reviews_and_calculate_price_then_return_data(filtered_restaurants_reviews_shuffled, percent=percent, reviews_num_threshold=reviews_num_threshold, genai=genai, sampling_method=sampling_method, column=column)                     

# df

In [ ]:
# # folder name
# folder_name = f"{percent}percent_{reviews_num_threshold}reviews_num_threshold_{genai}_{sampling_method}_{column}_research_data"

# current_directory = os.getcwd()
# parent_folder_path = os.path.abspath(os.path.join(current_directory, ".."))
# new_folder_path = os.path.join(parent_folder_path, folder_name)

# if not os.path.exists(new_folder_path):
#     os.makedirs(new_folder_path)
    
# new_subfolder_name = "original"
# new_subfolder_path = os.path.join(new_folder_path, new_subfolder_name)

# if not os.path.exists(new_subfolder_path):
#     os.makedirs(new_subfolder_path)

# df.to_csv(os.path.join("../", folder_name, new_subfolder_name, f"research_original_set_{percent}percent_{reviews_num_threshold}reviews_num_threshold_{genai}_{sampling_method}_{column}.csv"), index=False)

## Calculate all combination results (Sampling method is stratified)

In [8]:
# sampling by stratified
all_combination_results = []

percent_list = [i for i in range(1, 21)]
reviews_num_threshold_list = [i for i in range(1, 11)]
genai_list = ["GPT-3.5 Turbo"]
sampling_method_list = ['stratified']
column_list = ["stars"]

filtered_restaurants_reviews_shuffled = pd.read_csv("restaurants_reviews_shuffled.csv")

for percent in percent_list:
    for reviews_num_threshold in reviews_num_threshold_list:
        for genai in genai_list:
            for sampling_method in sampling_method_list:
                for column in column_list:
                    _, calculation_results = sample_reviews_and_calculate_price_then_return_data(filtered_restaurants_reviews_shuffled, percent=percent, reviews_num_threshold=reviews_num_threshold, genai=genai, sampling_method=sampling_method, column=column)
                    all_combination_results.append(calculation_results)

Percent: 1 %
Threshold reviews: 1
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 1 %
Threshold reviews: 2
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 1 %
Threshold reviews: 3
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 1 %
Threshold reviews: 4
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 1 %
Threshold reviews: 5
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 1 %
Threshold reviews: 6
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


------------

-----------------------------------------------------------


Percent: 5 %
Threshold reviews: 9
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 5 %
Threshold reviews: 10
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 6 %
Threshold reviews: 1
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 6 %
Threshold reviews: 2
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 6 %
Threshold reviews: 3
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 6 %
Threshold reviews: 4
Generative AI model: GPT-3.5

-----------------------------------------------------------


Percent: 10 %
Threshold reviews: 6
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 10 %
Threshold reviews: 7
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 10 %
Threshold reviews: 8
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 10 %
Threshold reviews: 9
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 10 %
Threshold reviews: 10
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 11 %
Threshold reviews: 1
Generative AI model: G

-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 3
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 4
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 5
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 6
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 7
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 15 %
Threshold reviews: 8
Generative AI model: GP

-----------------------------------------------------------


Percent: 19 %
Threshold reviews: 10
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 20 %
Threshold reviews: 1
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 20 %
Threshold reviews: 2
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 20 %
Threshold reviews: 3
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 20 %
Threshold reviews: 4
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------


Percent: 20 %
Threshold reviews: 5
Generative AI model: G

In [9]:
all_combination_results

[{'percent': 1,
  'reviews_num_threshold': 1,
  'sampling_method': 'stratified',
  'column': 'stars',
  'sampled_percent_reviews_num': 28170,
  'filtered_reviews_num': 28170,
  'filtered_users_count': 25901,
  'filtered_businesses_count': 13531,
  'min_reviews_per_user': 1,
  'mean_reviews_per_user': 1.0876027952588703,
  'max_reviews_per_user': 11,
  'min_reviews_per_business': 1,
  'mean_reviews_per_business': 2.0818860394649326,
  'max_reviews_per_business': 93,
  'genai': 'GPT-3.5 Turbo',
  'model_NT_price_per_token_input': 1.5e-05,
  'model_NT_price_per_token_output': 4.5e-05,
  'model_NT_price_per_token_input_and_output': 3e-05,
  'model_NT_price_per_picture': 0.2295,
  'avg_token_count_per_user': 121.16902822284854,
  'avg_token_count_per_business': 231.94139383637574,
  'price_per_user_by_output': 0.010905212540056369,
  'price_per_user_by_input_and_output': 0.007270141693370913,
  'price_for_all_user_by_output': 282.45591,
  'price_for_all_user_by_input_and_output': 188.30394,

In [10]:
all_combination_results_df = combination_results_list_to_df(all_combination_results)
all_combination_results_df

/var/folders/d7/xjcfv_7j38j7m33xhd0zpvzm0000gn/T/ipykernel_81484/709226683.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_combination_results_df = pd.concat([all_combination_results_df, pd.DataFrame([calculation_result])], ignore_index=True)


,percent,reviews_num_threshold,sampling_method,column,sampled_percent_reviews_num,filtered_reviews_num,filtered_users_count,filtered_businesses_count,min_reviews_per_user,mean_reviews_per_user,...,avg_token_count_per_user,avg_token_count_per_business,price_per_user_by_output,price_per_user_by_input_and_output,price_for_all_user_by_output,price_for_all_user_by_input_and_output,price_per_business_by_output,price_per_business_by_input_and_output,price_for_all_business_by_output,price_for_all_business_by_input_and_output
0,1,1,stratified,stars,28170,28170,25901,13531,1,1.087603,...,121.169028,231.941394,0.010905,0.007270,282.45591,188.30394,0.020875,0.013916,282.45591,188.30394
1,1,2,stratified,stars,28170,42,21,19,2,2.000000,...,253.333333,280.000000,0.022800,0.015200,0.47880,0.31920,0.025200,0.016800,0.47880,0.31920
2,1,3,stratified,stars,28170,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,stratified,stars,28170,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,stratified,stars,28170,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,20,6,stratified,stars,563384,48231,4706,3842,6,10.248831,...,1705.959201,2089.600208,0.153536,0.102358,722.54196,481.69464,0.188064,0.125376,722.54196,481.69464
196,20,7,stratified,stars,563384,17983,1626,1429,7,11.059656,...,1977.778598,2250.432470,0.178000,0.118667,289.42812,192.95208,0.202539,0.135026,289.42812,192.95208
197,20,8,stratified,stars,563384,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,20,9,stratified,stars,563384,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
all_combination_results_df.to_csv("../../research related documents/stratified_sampling_restaurant_reviews_combination_calculation_results.csv", index=False)

In [12]:
import pandas as pd

price_limit = 300
reviews_num = 5000
user_num = 500
business_num = 500

combination_calculation_results_df = pd.read_csv("../../research related documents/stratified_sampling_restaurant_reviews_combination_calculation_results.csv")

filter_combination_results(combination_calculation_results_df, 
                           price_limit, reviews_num, user_num, business_num)

,percent,reviews_num_threshold,sampling_method,column,sampled_percent_reviews_num,filtered_reviews_num,filtered_users_count,filtered_businesses_count,min_reviews_per_user,mean_reviews_per_user,...,avg_token_count_per_user,avg_token_count_per_business,price_per_user_by_output,price_per_user_by_input_and_output,price_for_all_user_by_output,price_for_all_user_by_input_and_output,price_per_business_by_output,price_per_business_by_input_and_output,price_for_all_business_by_output,price_for_all_business_by_input_and_output
0,1,1,stratified,stars,28170,28170,25901,13531,1.0,1.087603,...,121.169028,231.941394,0.010905,0.007270,282.45591,188.30394,0.020875,0.013916,282.45591,188.30394
21,3,2,stratified,stars,84508,10922,4311,3427,2.0,2.533519,...,382.165159,480.745258,0.034395,0.022930,148.27626,98.85084,0.043267,0.028845,148.27626,98.85084
31,4,2,stratified,stars,112676,21699,8037,5919,2.0,2.699888,...,392.661690,533.168103,0.035340,0.023560,284.02398,189.34932,0.047985,0.031990,284.02398,189.34932
41,5,2,stratified,stars,140845,34144,12001,8265,2.0,2.845096,...,403.553537,585.970478,0.036320,0.024213,435.87414,290.58276,0.052737,0.035158,435.87414,290.58276
52,6,3,stratified,stars,169015,9993,2388,2070,3.0,4.184673,...,704.993300,813.296618,0.063449,0.042300,151.51716,101.01144,0.073197,0.048798,151.51716,101.01144
62,7,3,stratified,stars,197185,18483,4226,3433,3.0,4.373639,...,703.381448,865.857850,0.063304,0.042203,267.52410,178.34940,0.077927,0.051951,267.52410,178.34940
72,8,3,stratified,stars,225355,28452,6196,4903,3.0,4.591995,...,721.249677,911.454824,0.064912,0.043275,402.19767,268.13178,0.082031,0.054687,402.19767,268.13178
83,9,4,stratified,stars,253521,7989,1387,1240,4.0,5.759913,...,1022.771449,1144.019355,0.092049,0.061366,127.67256,85.11504,0.102962,0.068641,127.67256,85.11504
93,10,4,stratified,stars,281692,15636,2554,2235,4.0,6.122161,...,1044.241973,1193.285906,0.093982,0.062655,240.02946,160.01964,0.107396,0.071597,240.02946,160.01964
103,11,4,stratified,stars,309862,25193,3998,3359,4.0,6.301401,...,1046.868184,1246.019351,0.094218,0.062812,376.68411,251.12274,0.112142,0.074761,376.68411,251.12274


In [13]:
import os
import pandas as pd

percent = 13
reviews_num_threshold = 5
genai = "GPT-3.5 Turbo"
sampling_method = 'stratified'
column = "stars"

filtered_restaurants_reviews_shuffled = pd.read_csv("restaurants_reviews_shuffled.csv")
df, calculation_results = sample_reviews_and_calculate_price_then_return_data(filtered_restaurants_reviews_shuffled, percent=percent, reviews_num_threshold=reviews_num_threshold, genai=genai, sampling_method=sampling_method, column=column)                     

df

Percent: 13 %
Threshold reviews: 5
Generative AI model: GPT-3.5 Turbo
Sampling_method: stratified
Column: stars


-----------------------------------------------------------




,review_id,user_id,business_id,stars,useful,funny,cool,text,date,token_count_nltk
7,m7ClRUTfNHP-Di8JOihQIw,2a9uyq3zsX5RIxG_jtpM4Q,TE2IEDNV0RcI6s1wTOP4fg,5,5,1,1,Unreal how good the tortillas and chips are fr...,2015-09-25 22:51:03,201
31,hH4j9sksZkq8L1Ioxu51xg,QSNzozwTwY9UU3QNvf-3jA,KDzLMPTTLihnpcctZLmg0g,5,4,1,1,Over the top amazing meal at the new Sea Salt ...,2015-03-15 02:26:01,528
61,8rO_5ohyo73oeJrRxYIVCw,5QRQJNdt3p6E7FWR9FEX3A,zuDXSlqm2veuwTD_Kl-pkw,5,0,0,0,I fell in love with this bread at the Garfield...,2017-09-01 03:03:59,116
71,V8fVosiAfVVtKsp9Qq855w,sGaoPWB_p1aOm5z9cnV3Aw,hX_8ZUmIqWFWzjdiPrf23w,5,0,0,0,"Oh em freaking gee, such great desserts and pa...",2017-02-08 01:56:09,75
139,mS7y9zKdplRKyCjSZSlzmg,v3jR1g-mGJVTTq7gurRN3A,_V6hl1oGkTV2KbGeax_HPA,5,0,0,0,I've been going here for years and it never di...,2015-12-15 00:45:59,60
...,...,...,...,...,...,...,...,...,...,...
365991,Z8axs93QQ0SJk11esMKZ1A,2dyfZNhtyuHdestczTgWjQ,sL6fC0P4C-gyL4E5gacUeQ,2,3,3,2,"Gourmet: This ain't it\nDry, doughy crust. Swe...",2010-03-28 21:17:37,120
366065,r4gDx4r8NPJzyVOT-CY00A,H-sUIQCG2B0pd8zwKjQsOA,EpREWeEpmR8f1qLHzzF0AA,2,1,0,1,The service was good. The beer was warm. The...,2015-06-23 18:39:52,77
366083,hecGF2TVQYBWXJ4kEUhDdw,dYzmK4IvlFHN-sUrsnyg9Q,5xW1QiUC2PEwa8jDha0scQ,2,10,3,3,I should really trust the yelp rating more. Wh...,2018-02-26 22:35:15,195
366092,eP2i-mnLqgAX-htmd8aUpg,IL0XURF7WCEo37HyJqtYGg,MI7cUsPiQGYQRuQmn3MbtA,2,3,2,2,"Again, I was let down here at Boca. To start o...",2015-01-11 23:17:55,338


In [14]:
# folder name
folder_name = f"{percent}percent_{reviews_num_threshold}reviews_num_threshold_{genai}_{sampling_method}_{column}_research_data"

current_directory = os.getcwd()
parent_folder_path = os.path.abspath(os.path.join(current_directory, ".."))
new_folder_path = os.path.join(parent_folder_path, folder_name)

if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)
    
new_subfolder_name = "original"
new_subfolder_path = os.path.join(new_folder_path, new_subfolder_name)

if not os.path.exists(new_subfolder_path):
    os.makedirs(new_subfolder_path)

df.to_csv(os.path.join("../", folder_name, new_subfolder_name, f"research_original_set_{percent}percent_{reviews_num_threshold}reviews_num_threshold_{genai}_{sampling_method}_{column}.csv"), index=False)

# Dataset split

## 13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars_research_data

In [30]:
def dataset_split(folder_name, file_name, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, stratify=(False, "stars")):
    
    import os
    import pandas as pd
    from sklearn.model_selection import train_test_split
    
    # Read the dataset
    df = pd.read_csv(os.path.join("../", folder_name, "original", file_name))
    
    if (train_ratio + valid_ratio + test_ratio) == 1 and train_ratio >= 0 and valid_ratio >= 0 and test_ratio >= 0:
        
        # Split the dataset
        if stratify[0]:
            # Split the dataset using stratified sampling
            train_data, remaining = train_test_split(df, test_size=(valid_ratio + test_ratio), stratify=df[stratify[1]], random_state=42)
            valid_data, test_data = train_test_split(remaining, test_size=test_ratio/(valid_ratio + test_ratio), stratify=remaining[stratify[1]], random_state=42)
        else:
            train_data, remaining = train_test_split(df, test_size=(valid_ratio + test_ratio), random_state=42)
            valid_data, test_data = train_test_split(remaining, test_size=test_ratio/(valid_ratio + test_ratio), random_state=42)
     
        # Reset index
        train_data.reset_index(drop=True, inplace=True)
        valid_data.reset_index(drop=True, inplace=True)
        test_data.reset_index(drop=True, inplace=True)
        
        print(f"Original dataset size: {df.shape}, ratio is {len(df)/len(df)}", )
        print(f"Training dataset size: {train_data.shape}, ratio is {len(train_data)/len(df)}")
        print(f"Validation dataset size: {valid_data.shape}, ratio is {len(valid_data)/len(df)}")
        print(f"Test dataset size: {test_data.shape}, ratio is {len(test_data)/len(df)}")
        
        return train_data, valid_data, test_data
    
    else:
        print("Please make sure each ratio is larger than 0 and sum of the three ratio is 1 !")

In [31]:
folder_name = "13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars_research_data"
file_name = "research_original_set_13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars.csv"

training_data, validation_data, test_data = dataset_split(folder_name, file_name, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, stratify=(False, "stars"))

Original dataset size: (13996, 10), ratio is 1.0
Training dataset size: (11196, 10), ratio is 0.7999428408116604
Validation dataset size: (1400, 10), ratio is 0.10002857959416976
Test dataset size: (1400, 10), ratio is 0.10002857959416976


In [32]:
print(training_data.shape)
print(training_data.head(), '\n')
print(validation_data.shape)
print(validation_data.head(), '\n')
print(test_data.shape)
print(test_data.head())

(11196, 10)
                review_id                 user_id             business_id  \
0  dviV-IHvxIH_aARChbQX3w  _Snvv_nGfpctBG4OOWwGLg  EFci0tbSrb7wko6tpVDnbA   
1  LFkOyBj-1hmx8UvLfm2iSw  WFP-b1Aep71xjS3TiGEC2Q  EPMHuTHu46B123ivRIZ-Xg   
2  23LB2UED00NnrTesgPdBkg  Y8uEQnccKOpgdYOE3ls1ug  J-ciDDEdIHMcChGIyKZnOg   
3  6jIhwgU_izeWlD_EF68CqQ  LYHQ_anXo-4MqYipQWsezA  teFjQxUqT8c-yxQdoILDVQ   
4  bGF02XnKW6gOGNoNMdEGfQ  XuaqlmZdRldcw97I_TtX9w  IzyQVIJG8JAnOiRQPb0-wg   

   stars  useful  funny  cool  \
0      1       4      7     2   
1      3       0      0     2   
2      5       2      0     3   
3      3       5      1     0   
4      3       6      6     4   

                                                text                 date  \
0  Ok obvi normally I'd dgaf about writing a revi...  2015-07-22 19:45:44   
1  Mmm don't see what the huge deal is about Magi...  2015-10-13 20:17:24   
2  Was looking for a specialty birthday cake that...  2019-06-23 15:33:50   
3  Food: A+\n\nSer

In [33]:
# save as csv

training_data.to_csv(os.path.join("../", folder_name, "original", "research_training_set.csv"), index=False)
validation_data.to_csv(os.path.join("../", folder_name, "original", "research_validation_set.csv"), index=False)
test_data.to_csv(os.path.join("../", folder_name, "original", "research_test_set.csv"), index=False)

In [ ]:
# # check if the distribution of stars column is the same (I ran this code, it's "almost" the same.)

# folder_name = "../13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars_research_data/original"

# train_data = pd.read_csv(os.path.join(folder_name, "research_training_set.csv"))
# valid_data = pd.read_csv(os.path.join(folder_name, "research_validation_set.csv"))
# test_data = pd.read_csv(os.path.join(folder_name, "research_test_set.csv"))
# original_set = pd.read_csv(os.path.join(folder_name, "research_original_set_13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars.csv"))

# print(train_data["stars"].value_counts()/len(train_data))
# print(valid_data["stars"].value_counts()/len(valid_data))
# print(test_data["stars"].value_counts()/len(test_data))
# print(original_set["stars"].value_counts()/len(original_set))

## 13percent_5reviews_num_threshold_GPT-3.5 Turbo_stratified_stars_research_data

In [35]:
folder_name = "13percent_5reviews_num_threshold_GPT-3.5 Turbo_stratified_stars_research_data"
file_name = "research_original_set_13percent_5reviews_num_threshold_GPT-3.5 Turbo_stratified_stars.csv"

training_data, validation_data, test_data = dataset_split(folder_name, file_name, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, stratify=(False, "stars"))

Original dataset size: (14449, 10), ratio is 1.0
Training dataset size: (11559, 10), ratio is 0.799986158211641
Validation dataset size: (1445, 10), ratio is 0.10000692089417952
Test dataset size: (1445, 10), ratio is 0.10000692089417952


In [36]:
# save as csv

training_data.to_csv(os.path.join("../", folder_name, "original", "research_training_set.csv"), index=False)
validation_data.to_csv(os.path.join("../", folder_name, "original", "research_validation_set.csv"), index=False)
test_data.to_csv(os.path.join("../", folder_name, "original", "research_test_set.csv"), index=False)

In [ ]:
# # check if the distribution of stars column is the same (I ran this code, it's "almost" the same.)

# folder_name = "../13percent_5reviews_num_threshold_GPT-3.5 Turbo_stratified_stars_research_data/original"

# train_data = pd.read_csv(os.path.join(folder_name, "research_training_set.csv"))
# valid_data = pd.read_csv(os.path.join(folder_name, "research_validation_set.csv"))
# test_data = pd.read_csv(os.path.join(folder_name, "research_test_set.csv"))
# original_set = pd.read_csv(os.path.join(folder_name, "research_original_set_13percent_5reviews_num_threshold_GPT-3.5 Turbo_random_stars.csv"))

# print(train_data["stars"].value_counts()/len(train_data))
# print(valid_data["stars"].value_counts()/len(valid_data))
# print(test_data["stars"].value_counts()/len(test_data))
# print(original_set["stars"].value_counts()/len(original_set))